## getting the data and all the necessary text files and functions

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [ ]:
#create a folder to store data
!mkdir sample_data/IAM
# copy the zip files that has data to this folder now
!cp -nav "/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips" /content/sample_data/IAM/

'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips' -> '/content/sample_data/IAM/words zips'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/l-m.zip' -> '/content/sample_data/IAM/words zips/l-m.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/g-h.zip' -> '/content/sample_data/IAM/words zips/g-h.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/a.zip' -> '/content/sample_data/IAM/words zips/a.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/n-p-r.zip' -> '/content/sample_data/IAM/words zips/n-p-r.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/e-f.zip' -> '/content/sample_data/IAM/words zips/e-f.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/j-k.zip' -> '/content/sample_data/IAM/words zips/j-k.zip'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words zips/c-d.zip' -> '/content/sample_data/IAM/words zips/c-d.zip'
'/conten

In [ ]:
# now unzip all the zips
!unzip -nq /content/sample_data/IAM/'words zips'/\*.zip -d /content/sample_data/IAM/'words zips'/


8 archives were successfully processed.


In [ ]:
# copy the ground truth texts file
!cp -navr "/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words.txt" "/content/sample_data/IAM/"

'/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/words.txt' -> '/content/sample_data/IAM/words.txt'


In [ ]:
data_location = '/content/sample_data/IAM/words zips'
words_txt_location = '/content/sample_data/IAM/words.txt'

In [ ]:
# copy the splits files
!cp -navr "/content/drive/My Drive/Colab Notebooks/OCR on IAM/train_files.txt" "/content/sample_data/IAM/"
!cp -navr "/content/drive/My Drive/Colab Notebooks/OCR on IAM/valid_files.txt" "/content/sample_data/IAM/"
!cp -navr "/content/drive/My Drive/Colab Notebooks/OCR on IAM/test_files.txt" "/content/sample_data/IAM/"

'/content/drive/My Drive/Colab Notebooks/OCR on IAM/train_files.txt' -> '/content/sample_data/IAM/train_files.txt'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/valid_files.txt' -> '/content/sample_data/IAM/valid_files.txt'
'/content/drive/My Drive/Colab Notebooks/OCR on IAM/test_files.txt' -> '/content/sample_data/IAM/test_files.txt'


In [ ]:
def get_paths_and_gts(partition_split_file):
    """
    read a string like(which can be found in each line of words.txt file):
    'a01-000u-00-00 ok 154 408 768 27 51 AT A'
    and extract 'a01-000u-00-00': location of the image with sub-folders, to read it from the directories
                'ok'            : processing status. ok means good, presumably.
                'A'             : ground truth text

    Then, pre-process using function defined above
    """
    # a list to store paths to images and ground truth texts
    paths_and_gts = []
    
    # open the file
    with open(partition_split_file) as f:
        # go through each line
        for line in f:
            # if a line is empty or commented with #, ignore that line
            if not line or line.startswith('#'):
                continue
            
            # in the text file, each line is seperated with '\n', so `strip` first
            # then string like 'a01-000u-00-00 ok 154 408 768 27 51 AT A' has to split to a list by spaces
            line_split = line.strip().split(' ')
            
            # the first item of the list contains path information, so split that by '-'
            directory_split = line_split[0].split('-')
            
            # now use all the above and concatenate to a string to make a path to an image
            image_location = f'{data_location}/{directory_split[0]}/{directory_split[0]}-{directory_split[1]}/{line_split[0]}.png'
            
            # in a string like 'a01-000u-00-00 ok 154 408 768 27 51 AT A', text from 9th split is the ground truth text.
            gt_text = ' '.join(line_split[8:])
            
            # ignore a sample(image and ground truth text), if the ground truth has more than 16 letters
            # if len(gt_text) > 16:
                # continue
            
            # now, append the image location and ground truth text of that image as a list to 
            paths_and_gts.append([image_location, gt_text])
    
    return paths_and_gts

## pre-processing

In [ ]:
def add_padding(img, old_w, old_h, new_w, new_h):
    h1, h2 = int((new_h - old_h) / 2), int((new_h - old_h) / 2) + old_h
    w1, w2 = int((new_w - old_w) / 2), int((new_w - old_w) / 2) + old_w
    img_pad = np.ones([new_h, new_w, 3]) * 255
    img_pad[h1:h2, w1:w2, :] = img
    return img_pad


def fix_size(img, target_w, target_h):
    h, w = img.shape[:2]
    if w < target_w and h < target_h:
        img = add_padding(img, w, h, target_w, target_h)
    elif w >= target_w and h < target_h:
        new_w = target_w
        new_h = int(h * new_w / w)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    elif w < target_w and h >= target_h:
        new_h = target_h
        new_w = int(w * new_h / h)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    else:
        """w>=target_w and h>=target_h """
        ratio = max(w / target_w, h / target_h)
        new_w = max(min(target_w, int(w / ratio)), 1)
        new_h = max(min(target_h, int(h / ratio)), 1)
        new_img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        img = add_padding(new_img, new_w, new_h, target_w, target_h)
    return img


def preprocess(path, img_w, img_h):
    """ Pre-processing image for predicting """
    img = cv2.imread(path)
    img = fix_size(img, img_w, img_h)

    img = np.clip(img, 0, 255)
    img = np.uint8(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = img.astype(np.float32)
    img /= 255
    return img

In [ ]:
letters = [' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?',
           'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
           'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
           'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_classes = len(letters) + 1
print(num_classes)

80


## generating all the pre-processed vectors

In [ ]:
def text_to_labels(text):
    return list(map(lambda x: letters.index(x), text))

def labels_to_text(labels):
    return ''.join(list(map(lambda x: letters[int(x)], labels)))

In [ ]:
train_files = get_paths_and_gts('/content/sample_data/IAM/train_files.txt')
valid_files = get_paths_and_gts('/content/sample_data/IAM/valid_files.txt')
test_files = get_paths_and_gts('/content/sample_data/IAM/test_files.txt')
len(train_files), len(valid_files), len(test_files)

(87292, 4316, 4316)

In [ ]:
# since this image is not readable is not readable, it has to be discarded.
# ../words/r06/r06-022/r06-022-03-05.png'

for index, (img_loc, gt_text) in enumerate(train_files):
    if 'r06-022-03-05' in img_loc:
        print(index)
    else:
        continue

4576


In [ ]:
train_files[4576]

['/content/sample_data/IAM/words zips/r06/r06-022/r06-022-03-05.png', 'more']

In [ ]:
del train_files[4576]
print(train_files[4576])

['/content/sample_data/IAM/words zips/g06/g06-042e/g06-042e-05-03.png', 'notice']


this is how images used for training will look like.

In [ ]:
i = 21
print(train_files[i][1])
an_img = preprocess(path=train_files[i][0], img_w=128, img_h=64)
plt.imshow(an_img.T)

In [ ]:
class TextImageGenerator:
    
    def __init__(self, data,
                 img_w,
                 img_h, 
                 batch_size, 
                 i_len,
                 max_text_len):
        
        self.img_h = img_h
        self.img_w = img_w
        self.batch_size = batch_size
        self.max_text_len = max_text_len
        self.samples = data
        self.n = len(self.samples)
        self.i_len = i_len
        self.indexes = list(range(self.n))
        self.cur_index = 0
        
    def build_data(self):
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts = []
        for i, (img_filepath, text) in enumerate(self.samples):
            img = preprocess(img_filepath, self.img_w, self.img_h)
            self.imgs[i, :, :] = img
            self.texts.append(text)
    
    def next_sample(self):
        self.cur_index += 1
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]
    
    def next_batch(self):
        while True:
            # width and height are backwards from typical Keras convention
            # because width is the time dimension when it gets fed into the RNN
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])
            Y_data = np.zeros([self.batch_size, self.max_text_len])
            input_length = np.ones((self.batch_size, 1)) * self.i_len
            label_length = np.zeros((self.batch_size, 1))
                                   
            for i in range(self.batch_size):
                img, text = self.next_sample()
                img = img.T
                img = np.expand_dims(img, -1)
                X_data[i] = img
                Y_data[i, :len(text)] = text_to_labels(text)
                label_length[i] = len(text)
                
            inputs = [X_data, Y_data, input_length, label_length]
            outputs = np.zeros([self.batch_size])
            yield (inputs, outputs)

In [ ]:
batch_size = 64
input_length = 30
max_text_len = 16
img_w = 128
img_h = 64

In [ ]:
train_data = TextImageGenerator(train_files, img_w, img_h, batch_size, input_length, max_text_len)
train_data.build_data()

In [ ]:
train_data.imgs.shape

(87291, 64, 128)

In [ ]:
validation_data = TextImageGenerator(valid_files, img_w, img_h, batch_size, input_length, max_text_len)
validation_data.build_data()

# making model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import backend as tf_keras_backend

tf_keras_backend.set_image_data_format('channels_last')
tf_keras_backend.image_data_format()

'channels_last'

In [ ]:
input_data = layers.Input(name='the_input', shape=(128,64,1), dtype='float32')  # (None, 128, 64, 1)

# Convolution layer (VGG)
iam_layers = layers.Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(2, 2), name='max1')(iam_layers)  # (None,64, 32, 64)

iam_layers = layers.Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(2, 2), name='max2')(iam_layers)

iam_layers = layers.Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(1, 2), name='max3')(iam_layers)  # (None, 32, 8, 256)

iam_layers = layers.Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.Conv2D(512, (3, 3), padding='same', name='conv6')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)
iam_layers = layers.MaxPooling2D(pool_size=(1, 2), name='max4')(iam_layers)

iam_layers = layers.Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)
iam_layers = layers.Activation('relu')(iam_layers)

# CNN to RNN
iam_layers = layers.Reshape(target_shape=((32, 2048)), name='reshape')(iam_layers)
iam_layers = layers.Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(iam_layers)

# RNN layer
# layer ten
iam_layers = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(iam_layers)
# layer nine
iam_layers = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(iam_layers)
iam_layers = layers.BatchNormalization()(iam_layers)

# transforms RNN output to character activations:
iam_layers = layers.Dense(80, kernel_initializer='he_normal', name='dense2')(iam_layers)
iam_outputs = layers.Activation('softmax', name='softmax')(iam_layers)

labels = layers.Input(name='the_labels', shape=[16], dtype='float32')
input_length = layers.Input(name='input_length', shape=[1], dtype='int64')
label_length = layers.Input(name='label_length', shape=[1], dtype='int64')


def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return tf_keras_backend.ctc_batch_cost(labels, y_pred, input_length, label_length)


# loss function
loss_out = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([iam_outputs, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, 128, 64, 1)] 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 64)  640         the_input[0][0]                  
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 64, 64)  256         conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 64, 64)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')
# you will know, why there aren't any metrics

In [ ]:
import time
from tensorflow.keras.callbacks import Callback

class EpochTimeHistory(Callback):
    """
    a custom callback to print the time(in minutes, to console) each epoch took during.
    """
    def on_train_begin(self, logs={}):
        self.train_epoch_times = []
        self.valid_epoch_times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        cur_epoch_time = round((time.time() - self.epoch_time_start)/60, 4)
        self.train_epoch_times.append(cur_epoch_time)
        print(" ;epoch {0} took {1} minutes.".format(epoch+1, cur_epoch_time))


    def on_test_begin(self, logs={}):
        self.test_time_start = time.time()

    def on_test_end(self, logs={}):
        cur_test_time = round((time.time() - self.test_time_start)/60, 4)
        self.valid_epoch_times.append(cur_test_time)
        print(" ;validation took {} minutes.".format(cur_test_time))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model_save_cb = ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-weights-epoch{epoch:02d}-val_loss{val_loss:.3f}.h5',
                                verbose=1, save_best_only=False, monitor='val_loss', save_weights_only=False)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='min')
# reduce_learning_rate_cb = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, cooldown=2, min_lr=0.00001, verbose=1)
epoch_times = EpochTimeHistory()

In [ ]:
batch_size, train_data.n, validation_data.n

(64, 87291, 4316)

In [ ]:
history_model_3 = model.fit(train_data.next_batch(),
                            validation_data=validation_data.next_batch(),
                            steps_per_epoch=train_data.n/batch_size,
                            validation_steps=validation_data.n // batch_size,
                            epochs=30,
                            callbacks=[earlystop, model_save_cb, epoch_times])

Epoch 1/30
1364/1363 [==============================] - ETA: 0s - loss: 7.3719 ;validation took 0.1656 minutes.

Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-weights-epoch01-val_loss3.449.h5
 ;epoch 1 took 7.2078 minutes.
1364/1363 [==============================] - 415s 304ms/step - loss: 7.3719 - val_loss: 3.4487
Epoch 2/30
1364/1363 [==============================] - ETA: 0s - loss: 2.2471 ;validation took 0.1339 minutes.

Epoch 00002: saving model to /content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-weights-epoch02-val_loss2.158.h5
 ;epoch 2 took 6.8968 minutes.
1364/1363 [==============================] - 414s 303ms/step - loss: 2.2471 - val_loss: 2.1582
Epoch 3/30
1364/1363 [==============================] - ETA: 0s - loss: 1.6207 ;validation took 0.1337 minutes.

Epoch 00003: saving model to /content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-weights-epoch03-val_loss1.643.h5
 ;epoch 3 took 6.8964 minutes.
1364/1363 [=

In [ ]:
model.save(filepath='/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-model-after-3rd-session.h5', overwrite=False, include_optimizer=True)

## plotting model statistics

In [ ]:
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
sns.set(rc={'figure.figsize':(14,7)})
plt.plot(history_model_3.history['loss'])
plt.plot(history_model_3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
plt.show()

In [ ]:
history_model_3.history.keys()

dict_keys(['loss', 'val_loss'])

In [ ]:
import pandas as pd

In [ ]:
loss_df = pd.DataFrame(data=history_model_3.history)
loss_df.head()

,loss,val_loss
0,7.371885,3.448700
1,2.247088,2.158155
2,1.620672,1.642994
3,1.319233,1.405329
4,1.094819,1.069993


In [ ]:
loss_df.sort_values(by=['loss', 'val_loss']).head()

,loss,val_loss
27,0.128097,0.299811
24,0.141549,0.348824
26,0.143362,0.421445
25,0.151138,4.603170
22,0.158717,26.978632


so, let's use the weights of 27th epoch.

## predictions on test set and calculating WER & CER

make predictions using weights from various checkpoints, since that's the epoch with lowest train and validation error.

In [ ]:
sns.reset_orig()
plt.figure(figsize=(3, 6))
i = 1
print(test_files[i][1])
temp_processed_image = preprocess(path=test_files[i][0], img_w=128, img_h=64)
plt.imshow(temp_processed_image.T)

In [ ]:
test_images_processed = []
original_test_texts = []
for _, (test_image_path, original_test_text) in enumerate(test_files):
     temp_processed_image = preprocess(path=test_image_path, img_w=128, img_h=64)
     test_images_processed.append(temp_processed_image.T)
     original_test_texts.append(original_test_text)

In [ ]:
print(len(test_files))
print(len(test_images_processed))
print(len(original_test_texts))

4316
4316
4316


In [ ]:
test_images_processed = np.array(test_images_processed)
test_images_processed.shape

(4316, 128, 64)

In [ ]:
test_images_processed = test_images_processed.reshape(4316, 128, 64, 1)
test_images_processed.shape

(4316, 128, 64, 1)

## with weights of final epoch (the one with the least loss on train data)

In [ ]:
iam_model_pred = None
iam_model_pred = Model(inputs=input_data, outputs=iam_outputs)
iam_model_pred.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 128, 64, 64)       640       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 64, 64)       256       
_________________________________________________________________
activation (Activation)      (None, 128, 64, 64)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 64, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 32, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 128)       512 

In [ ]:
iam_model_pred.load_weights(filepath='/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-model-after-3rd-session.h5')

In [ ]:
test_predictions_encoded = iam_model_pred.predict(x=test_images_processed)
test_predictions_encoded.shape

(4316, 32, 80)

In [ ]:
# use CTC decoder to decode to text
test_predictions_decoded = tf_keras_backend.get_value(tf_keras_backend.ctc_decode(test_predictions_encoded,
                                                                                  input_length = np.ones(test_predictions_encoded.shape[0])*test_predictions_encoded.shape[1],
                                                                                  greedy=True)[0][0])
test_predictions_decoded.shape

(4316, 14)

In [ ]:
def numbered_array_to_text(numbered_array):
    numbered_array = numbered_array[numbered_array != -1]
    return "".join(letters[i] for i in numbered_array)

In [ ]:
for i in range(10):
    print("original_text = ", original_test_texts[i])
    print("predicted text = ", numbered_array_to_text(test_predictions_decoded[i]))
    print()

original_text =  certain
predicted text =  ;certain

original_text =  biographies
predicted text =  ?biographies

original_text =  all
predicted text =  !&all

original_text =  It
predicted text =  It

original_text =  I
predicted text =  ?I

original_text =  show
predicted text =  #show

original_text =  Kings
predicted text =  XKings

original_text =  A
predicted text =  A

original_text =  the
predicted text =  7the

original_text =  and
predicted text =  !3and



### with the weight of epoch 24 (the one with the least loss on validation data)

In [ ]:
iam_model_pred = None
iam_model_pred = Model(inputs=input_data, outputs=iam_outputs)
iam_model_pred.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, 128, 64, 1)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 128, 64, 64)       640       
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 64, 64)       256       
_________________________________________________________________
activation (Activation)      (None, 128, 64, 64)       0         
_________________________________________________________________
max1 (MaxPooling2D)          (None, 64, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 32, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 128)       512 

In [ ]:
iam_model_pred.load_weights(filepath='/content/drive/My Drive/Colab Notebooks/OCR on IAM/data/two-weights-epoch24-val_loss0.242.h5')

In [ ]:
test_predictions_encoded = iam_model_pred.predict(x=test_images_processed)
# use CTC decoder to decode to text
test_predictions_decoded = tf_keras_backend.get_value(tf_keras_backend.ctc_decode(test_predictions_encoded,
                                                                                  input_length = np.ones(test_predictions_encoded.shape[0])*test_predictions_encoded.shape[1],
                                                                                  greedy=True)[0][0])
test_predictions_decoded.shape

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


(4316, 14)